# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


# Install detectron2

In [ ]:
!nvidia-smi
!nvcc -V

Fri Jun 25 07:36:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Update and purge Nvidia Driver
#### https://medium.com/analytics-vidhya/install-cuda-11-2-cudnn-8-1-0-and-python-3-9-on-rtx3090-for-deep-learning-fcf96c95f7a1

In [ ]:
!python -c "import torch; print(torch.__version__); import torchvision; print(torchvision.__version__)"

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
!pip install cmake
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# Manually install torch 1.8.1 as bug in 1.8.0
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install torchtext==0.9.1

#exit(0) # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.4MB/s eta 0:14:01tcmalloc: large alloc 1147494400 bytes == 0x55b3dac14000 @  0x7f8a0f4e4615 0x55b3a1825cdc 0x55b3a190552a 0x55b3a1828afd 0x55b3a1919fed 0x55b3a189c988 0x55b3a18974ae 0x55b3a182a3ea 0x55b3a189c7f0 0x55b3a18974ae 0x55b3a182a3ea 0x55b3a189932a 0x55b3a191ae36 0x55b3a1898853 0x55b3a191ae36 0x55b3a1898853 0x55b3a191ae36 0x55b3a1898853 0x55b3a191ae36 0x55b3a199d3e1 0x55b3a18fd6a9 0x55b3a1868cc4 0x55b3a1829559 0x55b3a189d4f8 0x55b3a182a30a 0x55b3a18983b5 0x55b3a18977ad 0x55b3a182a3ea 0x55b3a18983b5 0x55b3a182a30a 0x55b3a18983b5
     |█████████████████               | 1055.7MB 1.4MB/s eta 0:11:20tcmalloc: large alloc 1434370048 bytes == 0x55b41f26a000 @  0x7f8a0f4e4615 0x55b3a1825cdc 0x55b3a190552a 0x55b3a1828afd 0x55b3a1919fed 0x55b3a189c988 0x55b3a18974ae 0x55b3a182a3ea 0x55b3a189c7f0 0x55b3a18974ae 0x55b3a182a3ea 0x55b3a189932a 0x55b3a191ae36 0x55b3a1898853 0

In [ ]:
print(torch.__version__, torch.cuda.is_available())

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8.")   # need to manually install torch 1.8 if Colab changes its default version
!python -m pip install detectron2==0.4 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
#exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
!python -c 'import torch;print(torch.eye(3))'
!python -c 'from torch.utils.collect_env import main; main()'

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## FineTuning PubLayNet

In [ ]:
# download, decompress the data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NAgGrYoQJwdNXE-nFzx0yEJEoqxMcqh2' -O siemens.zip # exchange
!unzip siemens.zip -d datasets
!ls 'Siemens'

### Register Custom Dataset

In [ ]:
from detectron2.data.datasets import load_coco_json, register_coco_instances
from detectron2.data import MetadataCatalog

cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, "datasets/siemens")

# Training dataset
training_dataset_name = "dla_train"
training_json_file = os.path.join(data_dir, "train/train2021.json")
training_img_dir = os.path.join(data_dir, "train/images")

# Register custom training dataset
register_coco_instances(training_dataset_name, {}, training_json_file, training_img_dir)

# Validation dataset
validation_dataset_name = "dla_val"
validation_json_file = os.path.join(data_dir, "val/val2021.json")
validation_img_dir = os.path.join(data_dir, "val/images")
    
# Register custom training dataset
register_coco_instances(validation_dataset_name, {}, validation_json_file, validation_img_dir)
    
# Get training/validation dictionary
training_dict = load_coco_json(training_json_file, training_img_dir, dataset_name=training_dataset_name)
validation_dict = load_coco_json(validation_json_file, validation_img_dir, dataset_name=validation_dataset_name)

# Register metadata
training_metadata = MetadataCatalog.get(training_dataset_name)
validation_metadata = MetadataCatalog.get(validation_dataset_name)

In [ ]:
!git clone https://github.com/JaninaMattes/detectron2.git

### Get pre-trained model

In [ ]:
# download pre-trained model
!mkdir -p detectron2/pretrained_models/Resnet-101/
!wget https://www.dropbox.com/sh/wgt9skz67usliei/AADGw0h1y7K5vO0akulyXm-qa/model_final.pth?dl=0 -P /dev/null/detectron2/pretrained_models/Resnet-101/
!ls

### Train the pre-trained Mask-RCNN R50 FPN Model

In [ ]:
from detectron2.engine import DefaultTrainer

def setup(config_file, training_metadata):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.DATASETS.TRAIN = ("dla_train",)
    cfg.DATASETS.TEST = ("val_train")
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = "pretrained_models/Resnet-101/model_final.pth"
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
    cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for a toy dataset; you will need to train longer for a practical dataset
    cfg.SOLVER.STEPS = []        # do not decay learning rate
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # 128 faster, and good enough for a toy dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(training_metadata.thing_classes)  # has 5 classes (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
    
    # Solver options
    cfg.SOLVER.BASE_LR = 1e-3           # Base learning rate
    cfg.SOLVER.GAMMA = 0.5              # Learning rate decay
    cfg.SOLVER.STEPS = (250, 500, 750)  # Iterations at which to decay learning rate
    cfg.SOLVER.MAX_ITER = 1000          # Maximum number of iterations
    cfg.SOLVER.WARMUP_ITERS = 100       # Warmup iterations to linearly ramp learning rate from zero
    cfg.SOLVER.IMS_PER_BATCH = 1        # Lower to reduce memory usage (1 is the lowest)
    return cfg

cfg = setup(config_file="detectron2/configs/DLA_mask_rcnn_R_101_FPN_3x.yaml", training_metadata=training_metadata)

# Create output directory
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Start training
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Evaluation of model training

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_balloon_dicts("siemens_dataset/val")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

### Check metrics for COCO API 

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("dla_val", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "dla_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

### Download fine-tuned model

In [ ]:
files.download('fine_tunes_model.pth')